In [27]:
from pyspark.sql import SparkSession

In [28]:
spark = SparkSession.builder.appName('nlp').getOrCreate()

In [29]:
from pyspark.ml.feature import Tokenizer, RegexTokenizer
from pyspark.sql.functions import col,udf
from pyspark.sql.types import IntegerType

In [30]:
sen_df = spark.createDataFrame(data=[(0, 'Hi I heard about Spark'),
                                     (1, 'I wish java could use case classes'),
                                     (2, 'Logistic,regression,models,are,neat')], 
                               schema=['id', 'sentence'])

In [31]:
sen_df.show()

+---+--------------------+
| id|            sentence|
+---+--------------------+
|  0|Hi I heard about ...|
|  1|I wish java could...|
|  2|Logistic,regressi...|
+---+--------------------+



In [32]:
tokenizer = Tokenizer(inputCol='sentence', outputCol='words')
regex_tokenizer = RegexTokenizer(inputCol='sentence', outputCol='words', pattern='\\W')

In [33]:
count_tokens = udf(lambda words: len(words), IntegerType())

In [34]:
tokenized = tokenizer.transform(sen_df)
tokenized.show()

+---+--------------------+--------------------+
| id|            sentence|               words|
+---+--------------------+--------------------+
|  0|Hi I heard about ...|[hi, i, heard, ab...|
|  1|I wish java could...|[i, wish, java, c...|
|  2|Logistic,regressi...|[logistic,regress...|
+---+--------------------+--------------------+



In [44]:
tokenized.withColumn('tokens', count_tokens(col('words'))).show()

+---+--------------------+--------------------+------+
| id|            sentence|               words|tokens|
+---+--------------------+--------------------+------+
|  0|Hi I heard about ...|[hi, i, heard, ab...|     5|
|  1|I wish java could...|[i, wish, java, c...|     7|
|  2|Logistic,regressi...|[logistic,regress...|     1|
+---+--------------------+--------------------+------+



In [36]:
rg_tokenized = regex_tokenizer.transform(sen_df)
rg_tokenized.show()

+---+--------------------+--------------------+
| id|            sentence|               words|
+---+--------------------+--------------------+
|  0|Hi I heard about ...|[hi, i, heard, ab...|
|  1|I wish java could...|[i, wish, java, c...|
|  2|Logistic,regressi...|[logistic, regres...|
+---+--------------------+--------------------+



In [37]:
rg_tokenized.withColumn('tokens', count_tokens(col('words'))).show()

+---+--------------------+--------------------+------+
| id|            sentence|               words|tokens|
+---+--------------------+--------------------+------+
|  0|Hi I heard about ...|[hi, i, heard, ab...|     5|
|  1|I wish java could...|[i, wish, java, c...|     7|
|  2|Logistic,regressi...|[logistic, regres...|     5|
+---+--------------------+--------------------+------+



In [38]:
from pyspark.ml.feature import StopWordsRemover

In [39]:
sentenceDataFrame = spark.createDataFrame(data=[(0, ['I','saw','the','green','horse']),
                                                (1, ['Mary','had','a','little','lamb'])],
                                          schema=['id','tokens'])

In [40]:
sentenceDataFrame.show()

+---+--------------------+
| id|              tokens|
+---+--------------------+
|  0|[I, saw, the, gre...|
|  1|[Mary, had, a, li...|
+---+--------------------+



In [41]:
remover = StopWordsRemover(inputCol='tokens', outputCol='filtered')

In [42]:
remover.transform(sentenceDataFrame).show()

+---+--------------------+--------------------+
| id|              tokens|            filtered|
+---+--------------------+--------------------+
|  0|[I, saw, the, gre...| [saw, green, horse]|
|  1|[Mary, had, a, li...|[Mary, little, lamb]|
+---+--------------------+--------------------+



In [43]:
from pyspark.ml.feature import NGram

In [46]:
wordDataFrame = spark.createDataFrame(data=[(0, ['Hi' ,'I' ,'heard' ,'about' ,'Spark']),
                                     (1, ['I', 'wish', 'java', 'could', 'use', 'case', 'classes']),
                                     (2, ['Logistic','regression','models','are','neat'])], 
                               schema=['id', 'words'])

In [47]:
ngram = NGram(inputCol='words', outputCol='grams', n=2)

In [48]:
ngram.transform(wordDataFrame).select('grams').show(truncate=False)

+------------------------------------------------------------------+
|grams                                                             |
+------------------------------------------------------------------+
|[Hi I, I heard, heard about, about Spark]                         |
|[I wish, wish java, java could, could use, use case, case classes]|
|[Logistic regression, regression models, models are, are neat]    |
+------------------------------------------------------------------+



In [49]:
ngram_3 = NGram(inputCol='words', outputCol='grams', n=3)
ngram_3.transform(wordDataFrame).select('grams').show(truncate=False)

+--------------------------------------------------------------------------------+
|grams                                                                           |
+--------------------------------------------------------------------------------+
|[Hi I heard, I heard about, heard about Spark]                                  |
|[I wish java, wish java could, java could use, could use case, use case classes]|
|[Logistic regression models, regression models are, models are neat]            |
+--------------------------------------------------------------------------------+



In [52]:
from pyspark.ml.feature import HashingTF, IDF

In [53]:
sentenceData = spark.createDataFrame(data=[
                                    (0.0, "Hi I heard about Spark"),
                                    (0.0, 'I wish Java could use case classes'),
                                    (1.0, 'Logistic regression models are neat')],
                                    schema=['label', 'sentence'])

In [54]:
tokenizer = Tokenizer(inputCol='sentence', outputCol='words')

In [56]:
words_data = tokenizer.transform(sentenceData)
words_data.show(truncate=False)

+-----+-----------------------------------+------------------------------------------+
|label|sentence                           |words                                     |
+-----+-----------------------------------+------------------------------------------+
|0.0  |Hi I heard about Spark             |[hi, i, heard, about, spark]              |
|0.0  |I wish Java could use case classes |[i, wish, java, could, use, case, classes]|
|1.0  |Logistic regression models are neat|[logistic, regression, models, are, neat] |
+-----+-----------------------------------+------------------------------------------+



In [60]:
hashing_tf = HashingTF(inputCol='words', outputCol='rawFeatures')
featurized_data = hashing_tf.transform(words_data)
featurized_data.select(['rawFeatures']).show(truncate=False)

+--------------------------------------------------------------------------------------+
|rawFeatures                                                                           |
+--------------------------------------------------------------------------------------+
|(262144,[24417,49304,73197,91137,234657],[1.0,1.0,1.0,1.0,1.0])                       |
|(262144,[20719,24417,55551,116873,147765,162369,192310],[1.0,1.0,1.0,1.0,1.0,1.0,1.0])|
|(262144,[13671,91006,132713,167122,190884],[1.0,1.0,1.0,1.0,1.0])                     |
+--------------------------------------------------------------------------------------+



In [61]:
idf = IDF(inputCol='rawFeatures', outputCol='features')
idf_model = idf.fit(featurized_data)

In [67]:
rescaled_data = idf_model.transform(featurized_data)
rescaled_data.select(['features']).show(truncate=False)

+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|features                                                                                                                                                                                        |
+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|(262144,[24417,49304,73197,91137,234657],[0.28768207245178085,0.6931471805599453,0.6931471805599453,0.6931471805599453,0.6931471805599453])                                                     |
|(262144,[20719,24417,55551,116873,147765,162369,192310],[0.6931471805599453,0.28768207245178085,0.6931471805599453,0.6931471805599453,0.6931471805599453,0.6931471805599453,0.6931471805599453])|
|(262144,[13671,91006,132

In [68]:
from pyspark.ml.feature import CountVectorizer

# Input data: Each row is a bag of words with a ID.
df = spark.createDataFrame([
    (0, "a b c".split(" ")),
    (1, "a b b c a".split(" "))
], ["id", "words"])

# fit a CountVectorizerModel from the corpus.
cv = CountVectorizer(inputCol='words', outputCol='features', vocabSize=3, minDF=2.0)

model = cv.fit(df)

result = model.transform(df)
result.show(truncate=False)

+---+---------------+-------------------------+
|id |words          |features                 |
+---+---------------+-------------------------+
|0  |[a, b, c]      |(3,[0,1,2],[1.0,1.0,1.0])|
|1  |[a, b, b, c, a]|(3,[0,1,2],[2.0,2.0,1.0])|
+---+---------------+-------------------------+

